In [ ]:
import math

def get_float_input(prompt):
    while True:
        try:
            return float(input(prompt))
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

def compute_joint_shear(Ve, T1, T2):
    return T1 + T2 - Ve

def compute_tension_force(asteel, nbars, fy):
    '''
    asteel = area of steel, sq.mm
    nbars = no. of rebars, unitless
    fy = steel yield, MPa
    '''
    CONSTANT = 1.25
    n_to_kN = 1000
    results =  (CONSTANT * nbars * asteel * fy) / n_to_kN 
    return results

def compute_joint_area():
    '''
    beamw = beam width 
    jointd = joint depth 
    jointw = joint width
    pdistance = perpendicular distance
    ajoint = area of joint
    '''
    print("\n--- Compute Effective Joint Width and Aj ---")
    beamw = 500 
    jointd = 600 
    pdistance = 50

    ## Get minimum
    option_a = beamw + jointd
    option_b = beamw + (2 * pdistance)
    
    jointw = min(option_a, option_b)
    ajoint = jointw ** 2  # Square for effective area
    return jointw, ajoint

def compute_nominal_joint_shear_strength(LAMBDA, fc, ajoint, jointconfig):
    if jointconfig == 1:
        FACTOR = 1.7
    elif jointconfig == 2:
        FACTOR = 1.2
    else:
        FACTOR = 1.0
        
    n_to_kN = 1000
    LAMBDA = 1.0 #Normal-weight concrete
    results = (FACTOR * LAMBDA * math.sqrt(fc) * ajoint) / n_to_kN
    return results

def generate_report(Ve, T1, T2, Vjoint, jointw, ajoint, Vn, phiVn):
    report = f"""
    ---------- JOINT SHEAR CALCULATIONS REPORT ----------
    Probable Shear Strength Parameters:
    Ve (kN): {Ve:.2f}
    T1 (kN): {T1:.2f}
    T2 (kN): {T2:.2f}
    
    Computed Joint Shear:
    V_joint (kN): {Vjoint:.2f}
    
    Effective Joint Width and Area:
    Joint Width (mm): {jointw:.2f}
    Aj (mm^2): {ajoint:.2f}
    
    Nominal Joint Shear Strength:
    Vn (kN): {Vn:.2f}
    Phi * Vn (kN): {phiVn:.2f}
    
    Conclusion:
    {'No adjustment needed for shear reinforcement' if phiVn > Vjoint else 'Shear reinforcement adjustment required'}
    ------------------------------------------------------
    """
    return report

def main():
    print("\n--- Joint Shear Requirement for Special Moment Frames ---")
    
    # Inputs
    Ve = 384.23 #kN
    asteel1 = 491 #mm
    nbars1 = 8
    asteel2 = 491 #mm
    nbars2 = 4
    fy = 414
    CONSTANT = 1.25
    
    # Compute Tension Forces
    T1 = compute_tension_force(asteel1, nbars1, fy)
    T2 = compute_tension_force(asteel2, nbars2, fy)
    
    # Compute Joint Shear Force
    Vjoint = compute_joint_shear(Ve, T1, T2)
    
    # Compute Joint Area
    jointw, ajoint = compute_joint_area()
    
    # Joint Configuration Selection
    print("")
    print("Select Joint Configuration:")
    print("1 - Joints confined by beams on all four faces")
    print("2 - Joints confined by beams on three faces or two opposite faces")
    print("3 - Other cases")
    joint_config = int(get_float_input("Enter option (1, 2, or 3): "))
    
    # Compute Nominal Joint Shear Strength
    LAMBDA = 1.0  # Normal-weight concrete
    fc = 34.5 #MPa
    Vn = compute_nominal_joint_shear_strength(LAMBDA, fc, ajoint, joint_config)
    
    # Strength Reduction Factor
    phi = 0.85
    phiVn = phi * Vn
    
    # Generate and Print Report
    report = generate_report(Ve, T1, T2, Vjoint, jointw, ajoint, Vn, phiVn)
    print(report)
    
if __name__ == "__main__":
    main()